In [ ]:
from nvidia.dali.pipeline import Pipeline
import nvidia.dali.fn as fn
import nvidia.dali.types as types
import numpy as np
import os.path

In [ ]:
test_data_root = "/SORDI"

batch_size = 16

In [ ]:
pipe = Pipeline(batch_size=batch_size, num_threads=4, device_id=0)
with pipe:
    jpegs, bboxes, labels = fn.readers.coco(
        file_root="/",
        annotations_file="sordi.coco",
        #
        #polygon_masks=True,
        ratio=True)
    
    # image decompression, jpegs are done on GPU
    images = fn.decoders.image(jpegs, device="mixed", output_type=types.RGB) # mixed means input is CPU, output goes on GPU
    
    # Your augmentation can go here
    # Note, here num_threads is set to 4 CPU data loading threads. If more work is done here adapt it to CPU threads
    
    # You want those things to be prepared when calling pipe.run() later
    pipe.set_outputs(images, bboxes, labels) #, polygons, vertices)

In [ ]:
pipe.build()
pipe_out = pipe.run() # Note: Some of them are of type TensorListGPU, thus they require a .as_cpu() to get them visualized

In [ ]:
images_cpu = pipe_out[0].as_cpu()
bboxes_cpu = pipe_out[1]
labels_cpu = pipe_out[2]
#polygons_cpu = pipe_out[3]
#vertices_cpu = pipe_out[4]

In [ ]:
bboxes = bboxes_cpu.at(4)
labels = labels_cpu.at(4)

for bbox, label in zip(bboxes, labels):
    x, y, width, height = bbox
    print(f"Bounding box (x={x}, y={y}, width={width}, height={height}), label={label}")

"""
for polygon in polygons:
    mask_idx, start_vertex, end_vertex = polygon
    nvertices = end_vertex - start_vertex
    print(f"Polygon belonging to mask index {mask_idx} containing {nvertices} vertices:")
    polygon_vertices = vertices[start_vertex:end_vertex]
    for vertex_id in range(nvertices):
        x, y = vertices[vertex_id]
        print(f"Vertex {vertex_id}: x={x}, y={y}")
"""

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import random

In [ ]:
random.seed(12343)

In [ ]:
def plot_sample(img_index, ax):
    img = images_cpu.at(img_index)

    H = img.shape[0]
    W = img.shape[1]

    ax.imshow(img)
    bboxes = bboxes_cpu.at(img_index)
    labels = labels_cpu.at(img_index)
    
    categories_set = set()
    
    for label in labels:
        categories_set.add(label)

    category_id_to_color = dict([(cat_id , [random.uniform(0, 1), random.uniform(0, 1), random.uniform(0, 1)]) for cat_id in categories_set])

    for bbox, label in zip(bboxes, labels):
        rect = patches.Rectangle((bbox[0] * W,bbox[1] * H), bbox[2] * W,bbox[3] * H,
                                 linewidth=1, edgecolor=category_id_to_color[label], facecolor='none')
        ax.add_patch(rect)

In [ ]:
#visualize the first loads from above

fig, ax = plt.subplots(2, 2, figsize=(12, 12))
fig.tight_layout()
plot_sample(2, ax[0, 0])
plot_sample(1, ax[0, 1])
plot_sample(4, ax[1, 0])
plot_sample(8, ax[1, 1])
plt.show()

In [ ]:
#Let the pipeline run

for i in range(10):
    pipe_out = pipe.run()
    
    images_cpu = pipe_out[0].as_cpu()
    bboxes_cpu = pipe_out[1]
    labels_cpu = pipe_out[2]
    
    fig, ax = plt.subplots(2, 2, figsize=(12, 12))
    fig.tight_layout()
    plot_sample(2, ax[0, 0])
    plot_sample(1, ax[0, 1])
    plot_sample(4, ax[1, 0])
    plot_sample(8, ax[1, 1])
    plt.show()
    